# Modelo


En el mundo del análisis de datos y el modelado predictivo, una parte fundamental es comprender en detalle la naturaleza y el comportamiento de los datos con los que estamos trabajando ya que serán el insumo del entrenamiento del modelo. Esto se logra mediante un Análisis Exploratorio de Datos (EDA, por sus siglas en inglés), que nos permite examinar las características de nuestros datos, tanto numéricas como categóricas, y comprender su distribución, tendencias y posibles relaciones entre variables. Esta exploración inicial es crucial para tomar decisiones informadas sobre cómo proceder con el modelado y la predicción.

Además de la exploración de los datos, también nos proponemos seleccionar los modelos de clasificación más adecuados para nuestro conjunto de datos. Para lograr esto, evaluaremos el rendimiento de tres modelos ampliamente utilizados en la clasificación: Support Vector Machines (SVM), Random Forest Classifier y Gradient Boosting Trees. Estos modelos ofrecen enfoques diferentes para la clasificación y pueden adaptarse a una variedad de situaciones y tipos de datos. En este caso, para sacar la probabilidad de dicha clasificación, que al final de cuentas es el enfoque del proyecto

El proceso de selección de modelos implica entrenar y evaluar cada modelo utilizando métricas específicas, como la precisión (accuracy) y el recall. Además, exploraremos diferentes combinaciones de hiperparámetros para cada modelo, con el objetivo de encontrar la configuración óptima que maximice el rendimiento predictivo.

En resumen, este informe tiene como objetivo realizar un análisis exhaustivo de nuestros datos mediante un EDA detallado, seguido de la selección y evaluación de modelos de clasificación para predecir con precisión las etiquetas de clasificación, y posteriormente su probabilidad. Al finalizar, esperamos haber obtenido una comprensión profunda de nuestros datos y haber identificado el modelo o modelos que mejor se ajusten a nuestras necesidades de predicción.

## Importación de librerias

In [1]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC  # Para clasificación SVM
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn import tree

In [2]:
ruta_origen='C:/Users/Pc Com/Documents'
ruta_data = os.path.join(ruta_origen, 'final_transactions_dataset.csv')


In [3]:
df=pd.read_csv(ruta_data #,parse_dates=['date_BUY_fix','date_SELL_fix'],date_format='ISO8601'
               )
df=df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [4]:
df.head()

,company,sector,horizon (days),amount,date_BUY_fix,date_SELL_fix,price_BUY,price_SELL,Volatility_Buy,Volatility_sell,...,investment,ESG_ranking,PE_ratio,EPS_ratio,PS_ratio,PB_ratio,NetProfitMargin_ratio,current_ratio,roa_ratio,roe_ratio
0,BBY,RETAIL,2,100,2017-05-25,2017-05-26,55.551804,53.483913,0.383666,0.385748,...,BAD,12.0,12.58,3.73,0.38,3.19,3.01,1.49,8.69,26.69
1,BAC,BANK,330,15000,2016-11-22,2017-10-18,18.616749,24.654472,0.322809,0.236350,...,GOOD,26.3,11.39,1.26,1.71,0.54,15.70,0.92,0.67,5.54
2,AXP,BANK,7,3000,2016-09-27,2016-10-04,59.862297,59.517727,0.238642,0.235491,...,BAD,19.8,10.58,5.64,1.67,2.60,15.68,1.91,3.39,25.78
3,KSS,RETAIL,5,20000,2016-10-11,2016-10-17,38.216724,35.985329,0.428559,0.429340,...,BAD,12.9,11.09,3.27,0.36,1.25,3.17,1.60,4.41,11.35
4,JPM,BANK,360,15000,2015-03-12,2016-03-07,51.869335,52.047966,0.194612,0.254011,...,GOOD,27.9,9.38,5.46,1.87,0.81,19.91,0.99,0.81,8.91


In [5]:
df=df.sample(n=100,replace=False).reset_index()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   index                     100 non-null    int64  
 1   company                   100 non-null    object 
 2   sector                    100 non-null    object 
 3   horizon (days)            100 non-null    int64  
 4   amount                    100 non-null    int64  
 5   date_BUY_fix              100 non-null    object 
 6   date_SELL_fix             100 non-null    object 
 7   price_BUY                 100 non-null    float64
 8   price_SELL                100 non-null    float64
 9   Volatility_Buy            100 non-null    float64
 10  Volatility_sell           100 non-null    float64
 11  Sharpe Ratio              100 non-null    float64
 12  expected_return (yearly)  100 non-null    float64
 13  inflation                 100 non-null    float64
 14  nominal_ret

In [7]:
df.isnull().sum()

index                       0
company                     0
sector                      0
horizon (days)              0
amount                      0
date_BUY_fix                0
date_SELL_fix               0
price_BUY                   0
price_SELL                  0
Volatility_Buy              0
Volatility_sell             0
Sharpe Ratio                0
expected_return (yearly)    0
inflation                   0
nominal_return              0
investment                  0
ESG_ranking                 0
PE_ratio                    0
EPS_ratio                   0
PS_ratio                    0
PB_ratio                    0
NetProfitMargin_ratio       0
current_ratio               0
roa_ratio                   0
roe_ratio                   0
dtype: int64

In [8]:
independent_var_erase=['company','date_BUY_fix','date_SELL_fix']
dependent_var='investment'
independent_var=[x for x in df.columns if x not in independent_var_erase and x not in dependent_var]

In [9]:
X=df[independent_var]
y=df[dependent_var]

# Seleccionar columnas categoricas
VAR_categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]

# Seleccionar columnas numericas
VAR_numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

# Total de columnas
VAR_cols = VAR_categorical_cols + VAR_numerical_cols

print('Variables categoricas: ',VAR_categorical_cols)
print('Variables numericas: ',VAR_numerical_cols)
print('Variables independientes: ',VAR_cols)

Variables categoricas:  ['sector']
Variables numericas:  ['index', 'horizon (days)', 'amount', 'price_BUY', 'price_SELL', 'Volatility_Buy', 'Volatility_sell', 'Sharpe Ratio', 'expected_return (yearly)', 'inflation', 'nominal_return', 'ESG_ranking', 'PE_ratio', 'EPS_ratio', 'PS_ratio', 'PB_ratio', 'NetProfitMargin_ratio', 'current_ratio', 'roa_ratio', 'roe_ratio']
Variables independientes:  ['sector', 'index', 'horizon (days)', 'amount', 'price_BUY', 'price_SELL', 'Volatility_Buy', 'Volatility_sell', 'Sharpe Ratio', 'expected_return (yearly)', 'inflation', 'nominal_return', 'ESG_ranking', 'PE_ratio', 'EPS_ratio', 'PS_ratio', 'PB_ratio', 'NetProfitMargin_ratio', 'current_ratio', 'roa_ratio', 'roe_ratio']


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

In [11]:
# Definimos los preprocesadores:
# La funcion ColumnTransformer() nos permite aplicar otras funciones en un orden logico, y sobre qué columnas aplicarlas
# acá como ponemos ver aplicamos el codificacion numerica ordenada a las variables categoricas y escalamiento estandarizado
# a las numericas

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), VAR_categorical_cols)
        #,('num', StandardScaler(), VAR_numerical_cols)
    ],
    remainder='passthrough'
)

In [12]:
classifiers = {
    'SVC': SVC(),
    'RandomForestClassifier': RandomForestClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
}

In [13]:
# Paso 4: Define un diccionario de parámetros para cada clasificador
param_grid = {
    'SVC': {'classifier__C': [1, 10], 'classifier__kernel': ['linear', 'rbf'],'classifier__gamma':[ 1, 10]},
    'RandomForestClassifier': {'classifier__n_estimators': [50, 100], 'classifier__max_depth': [None, 10],'classifier__min_samples_split': [5, 10]},
    'GradientBoostingClassifier': {'classifier__n_estimators': [50, 100], 'classifier__learning_rate': [0.01, 0.1]}
}

In [14]:
# Paso 5: Define un diccionario de pipelines
pipelines = {}
for name, classifier in classifiers.items():
    pipelines[name] = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', classifier)
    ])

In [15]:
# Paso 6: Utiliza GridSearchCV para buscar los mejores parámetros
best_models = {}
training_accuracies = {}
for name, pipeline in pipelines.items():
    grid_search2 = GridSearchCV(pipeline, param_grid[name], cv=5,scoring='accuracy')
    grid_search2.fit(X_train, y_train)
    msg_training=f"accuracy en training -> {name}: best training score {grid_search2.best_score_}, modelo ganador {grid_search2.best_estimator_}"
    print(msg_training)
    best_models[name] = grid_search2.best_estimator_

accuracy en training -> SVC: best training score 0.675, modelo ganador Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OrdinalEncoder(),
                                                  ['sector'])])),
                ('classifier', SVC(C=0.1, gamma=0.1, kernel='linear'))])
accuracy en training -> RandomForestClassifier: best training score 0.975, modelo ganador Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OrdinalEncoder(),
                                                  ['sector'])])),
                ('classifier',
                 RandomForestClassifier(max_depth=20, min_samples_split=5,
                                        n_estimators=200))])
accuracy en training -> GradientBoostingClassifier: best training score 0.9875, modelo ganador Pipeline(steps=[('preprocessor',
   